In [5]:
import http.client
import json

# Setup the connection to your local server
conn = http.client.HTTPConnection("127.0.0.1", 1337)
headers = {'Content-Type': "application/json"}


In [6]:
context = '''

1
Representation Learning: A Review and New
Perspectives
Yoshua Bengio†, Aaron Courville, and Pascal Vincent†
Department of computer science and operations research, U. Montreal
† also, Canadian Institute for Advanced Research (CIFAR)
F
Abstract—
The success of machine learning algorithms generally depends on
data representation, and we hypothesize that this is because different
representations can entangle and hide more or less the different ex-
planatory factors of variation behind the data. Although specific domain
knowledge can be used to help design representations, learning with
generic priors can also be used, and the quest for AI is motivating
the design of more powerful representation-learning algorithms imple-
menting such priors. This paper reviews recent work in the area of
unsupervised feature learning and deep learning, covering advances
in probabilistic models, auto-encoders, manifold learning, and deep
networks. This motivates longer-term unanswered questions about the
appropriate objectives for learning good representations, for computing
representations (i.e., inference), and the geometrical connections be-
tween representation learning, density estimation and manifold learning.
Index Terms—Deep learning, representation learning, feature learning,
unsupervised learning, Boltzmann Machine, autoencoder, neural nets
1 INTRODUCTION
The performance of machine learning methods is heavily
dependent on the choice of data representation (or features)
on which they are applied. For that reason, much of the actual
effort in deploying machine learning algorithms goes into the
design of preprocessing pipelines and data transformations that
result in a representation of the data that can support effective
machine learning. Such feature engineering is important but
labor-intensive and highlights the weakness of current learning
algorithms: their inability to extract and organize the discrimi-
native information from the data. Feature engineering is a way
to take advantage of human ingenuity and prior knowledge to
compensate for that weakness. In order to expand the scope
and ease of applicability of machine learning, it would be
highly desirable to make learning algorithms less dependent
on feature engineering, so that novel applications could be
constructed faster, and more importantly, to make progress
towards Artificial Intelligence (AI). An AI must fundamentally
understand the world around us, and we argue that this can
only be achieved if it can learn to identify and disentangle the
underlying explanatory factors hidden in the observed milieu
of low-level sensory data.
This paper is about representation learning, i.e., learning
representations of the data that make it easier to extract useful
information when building classifiers or other predictors. In
the case of probabilistic models, a good representation is often
one that captures the posterior distribution of the underlying
explanatory factors for the observed input. A good representa-
tion is also one that is useful as input to a supervised predictor.
Among the various ways of learning representations, this paper
focuses on deep learning methods: those that are formed by
the composition of multiple non-linear transformations, with
the goal of yielding more abstract – and ultimately more useful
– representations. Here we survey this rapidly developing area
with special emphasis on recent progress. We consider some
of the fundamental questions that have been driving research
in this area. Specifically, what makes one representation better
than another? Given an example, how should we compute its
representation, i.e. perform feature extraction? Also, what are
appropriate objectives for learning good representations?
2 WHY SHOULD WE CARE ABOUT LEARNING
REPRESENTATIONS?
Representation learning has become a field in itself in the
machine learning community, with regular workshops at the
leading conferences such as NIPS and ICML, and a new
conference dedicated to it, ICLR1, sometimes under the header
of Deep Learning or Feature Learning. Although depth is an
important part of the story, many other priors are interesting
and can be conveniently captured when the problem is cast as
one of learning a representation, as discussed in the next sec-
tion. The rapid increase in scientific activity on representation
learning has been accompanied and nourished by a remarkable
string of empirical successes both in academia and in industry.
Below, we briefly highlight some of these high points.
Speech Recognition and Signal Processing
Speech was one of the early applications of neural networks,
in particular convolutional (or time-delay) neural networks 2.
The recent revival of interest in neural networks, deep learning,
and representation learning has had a strong impact in the
area of speech recognition, with breakthrough results (Dahl
et al., 2010; Deng et al., 2010; Seide et al., 2011a; Mohamed
et al., 2012; Dahl et al., 2012; Hinton et al., 2012) obtained
by several academics as well as researchers at industrial labs
bringing these algorithms to a larger scale and into products.
For example, Microsoft has released in 2012 a new version
of their MAVIS (Microsoft Audio Video Indexing Service)
1. International Conference on Learning Representations
2. See Bengio (1993) for a review of early work in this area.
arXiv:1206.5538v3 [cs.LG] 23 Apr 2014
2
speech system based on deep learning (Seide et al., 2011a).
These authors managed to reduce the word error rate on
four major benchmarks by about 30% (e.g. from 27.4% to
18.5% on RT03S) compared to state-of-the-art models based
on Gaussian mixtures for the acoustic modeling and trained on
the same amount of data (309 hours of speech). The relative
improvement in error rate obtained by Dahl et al. (2012) on a
smaller large-vocabulary speech recognition benchmark (Bing
mobile business search dataset, with 40 hours of speech) is
between 16% and 23%.
Representation-learning algorithms have also been applied
to music, substantially beating the state-of-the-art in poly-
phonic transcription (Boulanger-Lewandowski et al., 2012),
with relative error improvement between 5% and 30% on a
standard benchmark of 4 datasets. Deep learning also helped
to win MIREX (Music Information Retrieval) competitions,
e.g. in 2011 on audio tagging (Hamel et al., 2011).
Object Recognition
The beginnings of deep learning in 2006 have focused on
the MNIST digit image classification problem (Hinton et al.,
2006; Bengio et al., 2007), breaking the supremacy of SVMs
(1.4% error) on this dataset3. The latest records are still held
by deep networks: Ciresan et al. (2012) currently claims the
title of state-of-the-art for the unconstrained version of the task
(e.g., using a convolutional architecture), with 0.27% error,
and Rifai et al. (2011c) is state-of-the-art for the knowledge-
free version of MNIST, with 0.81% error.
In the last few years, deep learning has moved from
digits to object recognition in natural images, and the latest
breakthrough has been achieved on the ImageNet dataset4
bringing down the state-of-the-art error rate from 26.1% to
15.3% (Krizhevsky et al., 2012).
Natural Language Processing
Besides speech recognition, there are many other Natural
Language Processing (NLP) applications of representation
learning. Distributed representations for symbolic data were
introduced by Hinton (1986), and first developed in the
context of statistical language modeling by Bengio et al.
(2003) in so-called neural net language models (Bengio,
2008). They are all based on learning a distributed repre-
sentation for each word, called a word embedding. Adding a
convolutional architecture, Collobert et al. (2011) developed
the SENNA system5 that shares representations across the
tasks of language modeling, part-of-speech tagging, chunking,
named entity recognition, semantic role labeling and syntactic
parsing. SENNA approaches or surpasses the state-of-the-art
on these tasks but is simpler and much faster than traditional
predictors. Learning word embeddings can be combined with
learning image representations in a way that allow to associate
text and images. This approach has been used successfully to
build Google’s image search, exploiting huge quantities of data
to map images and queries in the same space (Weston et al.,
3. for the knowledge-free version of the task, where no image-specific prior
is used, such as image deformations or convolutions
4. The 1000-class ImageNet benchmark, whose results are detailed here:
http://www.image-net.org/challenges/LSVRC/2012/results.html
5. downloadable from http://ml.nec-labs.com/senna/
2010) and it has recently been extended to deeper multi-modal
representations (Srivastava and Salakhutdinov, 2012).
The neural net language model was also improved by
adding recurrence to the hidden layers (Mikolov et al., 2011),
allowing it to beat the state-of-the-art (smoothed n-gram
models) not only in terms of perplexity (exponential of the
average negative log-likelihood of predicting the right next
word, going down from 140 to 102) but also in terms of
word error rate in speech recognition (since the language
model is an important component of a speech recognition
system), decreasing it from 17.2% (KN5 baseline) or 16.9%
(discriminative language model) to 14.4% on the Wall Street
Journal benchmark task. Similar models have been applied
in statistical machine translation (Schwenk et al., 2012; Le
et al., 2013), improving perplexity and BLEU scores. Re-
cursive auto-encoders (which generalize recurrent networks)
have also been used to beat the state-of-the-art in full sentence
paraphrase detection (Socher et al., 2011a) almost doubling the
F1 score for paraphrase detection. Representation learning can
also be used to perform word sense disambiguation (Bordes
et al., 2012), bringing up the accuracy from 67.8% to 70.2%
on the subset of Senseval-3 where the system could be applied
(with subject-verb-object sentences). Finally, it has also been
successfully used to surpass the state-of-the-art in sentiment
analysis (Glorot et al., 2011b; Socher et al., 2011b).
Multi-Task and Transfer Learning, Domain Adaptation
Transfer learning is the ability of a learning algorithm to
exploit commonalities between different learning tasks in order
to share statistical strength, and transfer knowledge across
tasks. As discussed below, we hypothesize that representation
learning algorithms have an advantage for such tasks because
they learn representations that capture underlying factors, a
subset of which may be relevant for each particular task, as
illustrated in Figure 1. This hypothesis seems confirmed by a
number of empirical results showing the strengths of repre-
sentation learning algorithms in transfer learning scenarios.raw input x
task 1
output y 1
task 3
output y 3
task 2
output y 2
Task%A% Task%B% Task%C%
%output%
%input%
%shared%
subsets%of%

'''

task = '''

---

Task:
Summarize the authors, affiliations (company, university, other institution) and the keywords (if available or apparent from the abstract/ introduction) from the above text in the following format. Don't write code. I just need the output in this python-dictionary format so that I can use it in python in subsequent steps.

Example output:
{
"authors": ["Yan LeCun", "Jürgen Schmidthuber"],
"affiliations": ["NewYork University", "King Abdullah University of Science and Technology", "The Swiss AI Lab", "IDSIA"],
"keywords": "supervised learning", "classification"
}
'''

In [7]:
# Define the payload, specifying the "Mistral Instruct 7B Q4" model
payload = json.dumps({
    "messages": [
        {
            "content": "You are a helpful assistant.",
            "role": "system"
        },
        {
            "content": context + task,
            "role": "user"
        }
    ],
    "model": "mistral-ins-7b-q4",  # Specify the model here
    "stream": False,
    "max_tokens": 1000,
    "stop": [],
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "temperature": 0.1, # 0.7
    "top_p": 0.99, # 0.95
})


# Make the POST request
conn.request("POST", "/v1/chat/completions", payload, headers)

# Get the response
res = conn.getresponse()
data = res.read()

response = json.loads(data.decode("utf-8"))

# Print the response
# response
# response.keys()
# response["choices"][0].keys()
response_text = response["choices"][0]["message"]["content"]
print(response_text)

 {
"authors": ["Yoshua Bengio", "Aaron Courville", "Pascal Vincent"],
"affiliations": ["Department of computer science and operations research, U. Montreal", "Canadian Institute for Advanced Research (CIFAR)"],
"keywords": ["Deep learning", "representation learning", "feature learning", "unsupervised learning", "Boltzmann Machine", "autoencoder", "neural nets"]
}
